# Tensorflow intro

* [Официальная документация](https://www.tensorflow.org)
* [Get started](https://www.tensorflow.org/get_started/get_started)
* [Models](https://github.com/tensorflow/models/)

Из-за быстроты развития лучше всего смотреть на официальную документацию.

## Установка

Есть два типа установки: для CPU и GPU. Для CPU устанавливается стандартно через `pip`

Для GPU немного сложнее 
* проверить совместимость с видеокартой. Параметр CUDA Compute Capability должен быть больше 3.0
* Установить CUDA Toolkit восьмой версии
* Установить cuDNN версии 5.1
* Установить из pip пакет tensorflow-gpu

Можно установить через Docker. Можно и из исходников (говорят, что так рабоатет быстрее, потому что он не тянет кучу плюшек к себе).

## Базовые элементы

Самый простой код, с помощью которого легко убедиться, что все работает.

In [2]:
import tensorflow as tf # подключаем TF
hello = tf.constant('Hello, TensorFlow!') # создаем объект из TF
sess = tf.InteractiveSession() # создаем сессию
print(sess.run(hello)) #сессия "выполняет" объект

b'Hello, TensorFlow!'


## Граф

Вся работа с TF строится вокруг построения графа вычислений. Есть гарф - есть программа. По сути - это описание того, как будут проводиться вычисления. Основа TF - создать структуру, которая задаст порядок вычислений. Соответственно программа это: 
* составление графа вычислений 
* выполнение вычислений в структурах. 

Составляющие графа:
* Плейсхолдеры
* Переменные
* Операции
Из этого собираем граф, в котором будут вычисляться тензоры. **Тензоры** это многомерные массивы, по своей сути это топливо графа.Тензором может быть как отдельное число, вектор, так и целый батч.Вместо одного объекта можно передать массив объектов - на выходе получить массив ответов. По сути это обработка массива в numpy.

## Сессия

Выполнение графа происходит в сессии (*tf.Session()*). Такой объект скрывает в себе контекст выполнения графа (ресурсы, классы, адресные пространства). Два типа сессии:
* Обычные (**tf.Session()**)
* Интерактивные (**tf.InteractiveSession()**)
Интерактивная для консоли. Основной эффект — объект сессии не нужно передавать в функции вычисления как параметр.

Tensorboard генерирует графы за вас на примере:

<img src="pic/1.png" />

## Тензоры, операции, переменные

Тензор с нулями:

In [3]:
zeros_tensor = tf.zeros([3, 3])

Для того, чтобы посмотреть тензор, необходимо выполнить его. Пока без графа:

In [5]:
print(zeros_tensor.eval())
print(zeros_tensor)

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
Tensor("zeros:0", shape=(3, 3), dtype=float32)


Различия: в первой строке вычисление, во второй представление объекта. Что дает описание:
* Имя тензора
* Форма тензора (размерность массива numpy)
* Типизация тензора

Есть множество операций:

In [9]:
a = tf.truncated_normal([2, 2]) #усеченое нормальное распределение
b = tf.fill([2, 2], 0.5) #массив из 0.5 
print(sess.run(a + b)) 
print(sess.run(tf.matmul(a, b)))

[[ 0.45460051  0.46388394]
 [-0.28661668  1.51374352]]
[[-1.01343989 -1.01343989]
 [-0.17550369 -0.17550369]]


** sess.run()** метод исполнения операций в графе. 

Создадим переменную на основе тензора

In [10]:
v = tf.Variable(zeros_tensor)

Почему нельзя просто задать? Так как переменная будет в качестве узла графа. Есть еще плейсхолдеры - параметризуют граф и отмечают места для постановки нужных значений (внешних) по сути - обещание поставить значение потом.

In [11]:
x = tf.placeholder(tf.float32, shape = (4, 4)) 

In [32]:
a = tf.placeholder("float")
b = tf.placeholder("float")
y = tf.multiply(a, b)
print(sess.run(y, feed_dict={a:100, b:500}))

50000.0


## Basic eval

In [33]:
x = tf.placeholder(tf.float32)
f = 1 + 2 * x + tf.pow(x, 2)
sess.run(f, feed_dict={x:10})

121.0

<img src="pic/2.png" />

In [16]:
import numpy as np
x = tf.placeholder(dtype=tf.float32)
sigma = 1 / (1 + tf.exp(-x))
sigma.eval(feed_dict={x: np.linspace(-5, 5) })

array([ 0.00669285,  0.00819568,  0.01003255,  0.01227603,  0.01501357,
        0.01835024,  0.02241159,  0.02734679,  0.03333168,  0.04057176,
        0.04930425,  0.05979915,  0.07235796,  0.0873094 ,  0.10500059,
        0.12578245,  0.14998817,  0.17790413,  0.20973383,  0.24555731,
        0.28529069,  0.32865256,  0.3751457 ,  0.42406148,  0.47451192,
        0.52548808,  0.57593852,  0.62485433,  0.67134744,  0.71470934,
        0.75444269,  0.79026616,  0.82209587,  0.85001183,  0.87421757,
        0.89499938,  0.91269064,  0.92764211,  0.94020081,  0.95069569,
        0.95942819,  0.96666825,  0.97265327,  0.97758842,  0.98164982,
        0.98498636,  0.98772395,  0.98996747,  0.99180436,  0.99330717], dtype=float32)

<img src="pic/3.png" />

В фрагменте с запуском вычисления функции есть один момент, который отличает этот пример от предыдущих. Дело в том, что в плейсхолдер вместо одного скалярного значения мы передаем целый массив. TF обрабатывает все значения массива вместе, в рамках одного тензора (помним, что массив == тензор). Точно таким же образом мы можем передавать в граф объекты целыми батчами и поставлять нейронной сети картинки целиком.

## Сохранение и загрузка графов

Есть специальный объект-сериализатор, который делает две вещи:
1. Сохраняет текущий граф, состояние и значения в файл
2. Читает то же самое из файла

https://www.tensorflow.org/programmers_guide/variables

In [35]:
saver = tf.train.Saver()
saver.save(sess, '"test.ckpt"')

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: save_8/SaveV2 = SaveV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_8/Const_0, save_8/SaveV2/tensor_names, save_8/SaveV2/shape_and_slices, Variable)]]

Caused by op 'save_8/SaveV2', defined at:
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().s

In [37]:
ckpt = tf.train.get_checkpoint_state(ckpt_dir)
if ckpt and ckpt.model_checkpoint_path:
    print(ckpt.model_checkpoint_path)
    saver.restore(session, ckpt.model_checkpoint_path)

NameError: name 'ckpt_dir' is not defined

## Tensorboard

Крайне полезная система в составе TF — web-dashboard, который позволяет собирать статистику из дампов и логов и наблюдать, что же всё-таки происходит во время вычислений. Крайне удобно то, что дашборд работает на веб-сервере и можно, например, запустив tensorboard на удаленной машине в облаке, наблюдать происходящее у себя в окне браузера.

Tensorboard умеет:

1. Рисовать граф вычислений.
2. Граф вычислений стоит посмотреть хотя бы для самопроверки, чтобы убедиться в том, что собралось и считается именно то, что планировалось, и при кодировании не допущено ошибок.
3. Показывать статистику по переменным.
4. Можно собирать вообще любую статистику.
5. Есть средство для анализа многомерных данных (например, эмбеддингов). Для этого в дашборде встроены PCA и t-SNE, с которыми можно попробовать рассмотреть данные в 2 и 3 измерениях.
6. Гистограммы. Можно строить гистограммы распределений выходов слоев сетей и поведения переменных.

Обратная сторона медали — чтобы статистика попадала в дашборд, её нужно сохранять в логи (в формате protobuf) с помощью специального API. API не очень сложный, сгруппирован в **tf.summary**. Для сбора статистики нужно будет отдельно зарегистрировать интересующие переменные с помощью специальных функций и потом отдельно сохранить всё в лог.

In [38]:
tf.summary.histogram("layer_output", v)

<tf.Tensor 'layer_output:0' shape=() dtype=string>

**tf.summary.scalar("accuracy", learning_rate)**

Сохранять логи

In [39]:
writer = tf.summary.FileWriter("./logs/nn_logs", sess.graph) # for 1.0
merged = tf.summary.merge_all()

Для простого: 

In [41]:
import os
merged = tf.summary.merge_all(key='summaries')
if not os.path.exists('tensorboard_logs/'):
    os.makedirs('tensorboard_logs/')
my_writer = tf.summary.FileWriter('tensorboard_logs/', sess.graph)

По умолчанию Tensorboard локально доступен по адресу 127.0.1.1:6006. **`tensorboard --logdir=path/to/log-directory`**

## Apache beam and tf tranform

* TF: https://github.com/tensorflow/transform
* Beam: https://beam.apache.org/documentation/programming-guide/

**Apache Beam**.Это набор интерфейсов для создания data processing pipeline. Вы пишете программу с помощью этих интерфейсов, а потом запускаете ее на конкретном движке, будь это Apache Spark или Google Cloud DataFlow. 

### Install

Apache Beam только для Python 2.7

### tr.transform

**tf.Transform** библиотека для выполнения предварительной обработки данных с TensorFlow. Он позволяет комбинировать различные фреймворки обработки данных (Apache Beam). Удобно, что можно включать это в сам граф Tensorflow (можно увидеть сколько занимал препроцессинг данных, например). Или можно экспортировать граф обработки данных, чтобы препроцессинг и обучение модели были более менее независимыми. 

#### Intro transform

Несмотря на то, что TF позволяет работать с батчами данных, для некоторых операций необходим полный проход по датасету.Например, нормализация.

#### Определение функции Preprocessing

Опишем, как определить «preprocessing function», которая является логическим описанием pipline, который преобразует исходные данные в данные, которые будут использоваться для обучения модели ML. 

Набор данных - словарь столбцов, а функция предварительной обработки определяется двумя основными механизмами:
1. **tf.map**: принимает определенную пользователем функцию, которая принимает и возвращает тензоры. Такая функция может использовать любую операцию TensorFlow для построения выходных тензоров (от входных). Остальные аргументы - столбцы, к которым применяется функция. Количество столбцов = количеству аргументов функции. Аналог `map` в Python. Каждая строка обрабатывается независимо друг от друга.
2. **analyzers**: анализаторы являются функциями, которые принимают один или несколько колонок и ворзвращает некоторые сводные статистические данные для входного столбца или столбцов. Пример анализатора **tft.min**.

Объединив анализаторы и tft.map пользователи могут гибко создавать pipline для преобразования данных. Следующая функция предварительной обработки преобразует каждый из трех колонок по-разному.

In [1]:
import tensorflow as tf
import tensorflow_transform as tft

def preprocessing_fn(inputs):
  x = inputs['x']
  y = inputs['y']
  s = inputs['s']
  x_centered = tft.map(lambda x, mean: x - mean, x, tft.mean(x))
  y_normalized = tft.scale_to_0_1(y)
  s_integerized = tft.string_to_int(s)
  x_centered_times_y_normalized = tft.map(lambda x, y: x * y,
                                          x_centered, y_normalized)
  return {
      'x_centered': x_centered,
      'y_normalized': y_normalized,
      'x_centered_times_y_normalized': x_centered_times_y_normalized,
      's_integerized': s_integerized
  }

* `x, y, s` входные столбцы
* Первый новый столбец **x_centered**, строится путем составления `tft.map и tft.mean`. **tft.mean(x)** возвращает статистическую величину, представляющую среднее значение столбца x.
* Второй новый столбец **y_normalized**, созданный таким же образом, но с использованием методы **tft.scale_to_0_1**.
* Колонка **s_integerized** показывает пример работы со строками. В этом простом случае мы берем строку и переводим в целое число.
* **x_centered_times_y_normalized** объединенная колонка.

Типичные рабочий процесс пользователя tf.Transform будет построить функцию предварительной обработки, а затем включить это в больший pipline Beam (материализует данные для обучения).


#### Batching

Хотя это не очевидно из приведенного выше примера, заданная пользовательская функция передается tft.map будет передаваться тензорами, представляющих батчи, а не отдельные экземпляры, так же, как будет происходить во время подготовки TensorFlow.

### Реализация Canonical Beam

**tf.Transform** обеспечивает каноническую реализацию препроцессинга данных, которая запускает функцию предварительной обработки на Apache Beam.

Содержание **transformed_dataset** будет показано ниже, он содержать преобразованные столбцы в том же формате, что и исходные данные. В частности, значения s_integerized = `[0, 1, 0]`(эти значения зависят от того, как слова hello и world были нанесены на карту в целые числа, который является детерминированным). Для столбца x_centered мы вычитали среднее значение, поэтому значения столбца x,  `[1.0, 2.0, 3.0]` стали `[-1.0, 0.0, 1.0]`. Точно так же все остальные столбцы соответствуют их ожидаемым значениям.

Оба **raw_data** и **transformed_data** являются наборами данных. Другое возвращаемое значение, **transform_fn** является представлением преобразования, которое было сделано с данными (которые мы обсудим более подробно ниже).

В самом деле, **AnalyzeAndTransformDataset** представляет собой композицию из двух фундаментальных преобразований , предусмотренных реализации, **AnalyzeDataset** и **TransformDataset**. То есть, эти два фрагмента кода ниже эквивалентны.

**transform_fn** является чистой функцией, которая представляет собой операцию, применяемая к каждой строке набора данных. В частности, все значения анализатора уже вычислены и рассматриваются как константы. В нашем примере, transform_fn будет содержать в качестве констант среднее значение столбца x, min/max колонны y и словарь, используемый для отображения строки в целые числа.


Ключевая особенность tf.Transform является то , что transform_fnпредставляет собой map над rows (чистая функция, применяемая к каждой строке отдельно). Все вычисления с участием агрегирования строк делается **AnalyzeDataset**. Кроме того, transform_fn представляются в виде TensorFlow Graph, что означает, что он может быть встроен в обслуживающий граф.

Google предлагает **AnalyzeAndTransformDataset** для того, чтобы обеспечить оптимизацию. Это аналоги scikit-learn, который обеспечивает fit, transformи fit_transform методы для препроцессоров.

### Форматы данных и схемы

Выше мы упустили код для **raw_data_metadata**. В метаданных храниться схема, которая определяет расположение данных таким образом, что она может быть записана и прочитана в различные форматы (описание ниже).

1. **dataset_schema.Schema** класс представляет собой оболочку вокруг Dict из **dataset_schema.ColumnSchema**. Каждый ключ в cловаре описывает имя тензора, ColumnSchema описывает вид тензора и как он представлен в памяти или на диске.
2. Первый аргумент **ColumnSchema** определяет, Domain который включает в себя тип данных и некоторые детали, такие как диапазоны. В нашем случае мы указали только тип данных.
3. Второй аргумент содержит список **axis** объектов, которые описывают форму тензора. В нашем случае нет осей, так как это скаляры (rank 0 tensor).
4. Третий аргумент является представлением данных. Есть три вида представления: 
    *  **FixedColumnRepresentation** является представлением колонки с фиксированным, известного размером.
    * **ListColumnRepresentation** представляет колонки с разным размером
    * **SparseColumnRepresentation** колонки с фиксированным размером (sparse  представление).
    * подробнее [tf_metadata/dataset_schema.py](https://github.com/tensorflow/transform/blob/master/tensorflow_transform/tf_metadata/dataset_schema.py)

### IO with BEAM implementation

1. Строим схему данных
2. `csv_coder.CsvCoder`. Используем схему для чтения данных .csv. `ordered_columns` содержит названия всех колонок в порядке их появления в файле (сама схема не содержит эту информацию). 
3. Далее производим чтение из файла >> производим Map - декодирование

**raw_data** Переменная представляет собой PCollection содержащий данные в том же формате, что и список raw_data из предыдущего примера, а также использование из AnalyzeAndTransformDataset преобразования одно и то же. Обратите внимание , что схема используется в двух местах: чтение данных из файла CSV, а также в качестве вклада AnalyzeAndTransformDataset. Это происходит потому , что как формат CSV и формат , в памяти должны быть соединены со схемой для того , чтобы интерпретировать их как тензоры.